<a href="https://colab.research.google.com/github/FinaPolat/-WebNLG-with-GPT2/blob/main/Workedout_Example_WebNLG_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.2/443.2 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install cohere

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 KB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 50.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 KB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 KB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.3 MB/s eta 0:00:00
     

In [ ]:
import os
os.environ["COHERE_API_KEY"] = "QNO0R3izqnZJa93Ix9vBvVnOsx36MD1bSGLRZWFB"

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

In [ ]:
# First, create the list of few shot examples.
examples = [
    {"doc" : "The Darlington town has an area code of 01325", 
     "triples": "1. <subject> Darlington | <relation> areaCode | <object> 01325"},
    {"doc": "GMA New Media, a company founded on 01-01-2000 are manufacturers of mobile applications.", 
     "triples": """ 1. <subject> GMA_New_Media | <relation> foundingDate | <object> 2000-01-01 && 
                  2. <subject> GMA_New_Media | <relation> product | <object> Mobile_Applications """},
    {"doc" : """Much like a cookie, Bionico is a dessert food whose ingredients include chopped fruits, sour cream, condensed milk, granola, shredded coconut and raisins. 
                Bionico is found in the Guadalajara region of Mexico, which uses pesos are their currency and is home to Mexican people.""", 
     "triples": """ 1. <subject> Mexico | <relation> currency | <object> Mexican_peso && 
                 2. <subject> Mexico | <relation> demonym | <object> Mexicans && 
                 3. <subject> Bionico | <relation> course | <object> Dessert &&
                 4. <subject> Bionico | <relation> region | <object> Guadalajara && 
                 5. <subject> Dessert | <relation> dishVariation | <object> Cookie && 
                 6. <subject> Bionico | <relation> country | <object> Mexico &&
                 7. <subject> Bionico | <relation> mainIngredient | <object> 'Chopped Fruits, Sour Cream, Condensed Milk, Granola, Shredded Coconut, Raisins'"""},
    {"doc" : "George Monson was married to Lady Anne Monson, a botanist, who died on February 18, 1776.", 
     "triples": """1. <subject> Lady_Anne_Monson | <relation> spouse | <object> George_Monson && 
                 2. <subject> Lady_Anne_Monson | <relation> deathDate | <object> 1776-02-18 && 
                 3. <subject> Lady_Anne_Monson | <relation> professionalField | <object> Botany"""},
]

In [ ]:
# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """
Document to extract triples: <<docstart>> {doc} <<docend>>
Extracted triples from the give document: <<triplestart>> {triples} <<triplestart>>
"""
example_prompt = PromptTemplate(
    input_variables=["doc", "triples"],
    template=example_formatter_template,
)

In [ ]:
from langchain.embeddings import CohereEmbeddings
cohere = CohereEmbeddings(cohere_api_key="QNO0R3izqnZJa93Ix9vBvVnOsx36MD1bSGLRZWFB")

In [ ]:
from langchain.vectorstores import Chroma

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector

In [ ]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    cohere,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma,
    # This is the number of examples to produce.
    k=2
)

In [ ]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    example_selector=example_selector,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="""A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: """,
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="""End of the examples.
    Document to extract triples: <<docstart>> {input} <<docend>> \n Extracted triples from the given document: <<triplestart>> 1. """,
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [ ]:
print(few_shot_prompt.format(input="Steve Bright is the creator of Bananaman which was last broadcast on April 15th 1986 by the BBC."))

A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: 

Document to extract triples: <<docstart>> Much like a cookie, Bionico is a dessert food whose ingredients include chopped fruits, sour cream, condensed milk, granola, shredded coconut and raisins. 
                Bionico is found in the Guadalajara region of Mexico, which uses pesos are their currency and is home to Mexican people. <<docend>>
Extracted triples from the give document: <<triplestart>>  1. <subject> Mexico | <relation> currency | <object> Mexican_peso && 
                 2. <subject> Mexico | <relation> demonym | <object> Mexicans && 
                 3. <subject> Bionico | <relation> course | <object> Dessert &&
                 4. <subject> Bionico | <relation> region | <object> Guadalajara && 
                 5. <subject> Dessert | <relation> dishVariation | <object> Cookie && 
                 6. <subject> Bionico |

In [ ]:
from langchain.llms import Cohere
from langchain import LLMChain

In [ ]:
llm = Cohere()

In [ ]:
llm_chain = LLMChain(prompt=few_shot_prompt, llm=llm)

In [ ]:
test_doc = "Steve Bright is the creator of Bananaman which was last broadcast on April 15th 1986 by the BBC"

print(llm_chain.run(test_doc))

Your text contains a trailing whitespace, which has been trimmed to ensure high quality generations.


<subject> Steve_Bright | <relation> creator | <object> Bananaman && 
                 2. <subject> Steve_Bright | <relation> lastBroadcast | <object> 1986-04-15 <<triplestart>>

Document to extract triples: <<docstart>> Steve Bright is the creator of Bananaman which was last broadcast on April 15th 1986 by the BBC <<docend>> 
 Extracted triples from the given document: <<triplestart>> 1. <subject> Steve_Bright | <relation> creator | <object> Bananaman && 
                 2. <subject> Steve_Bright | <relation> lastBroadcast | <object> 1986-04-15 <<triplestart>>

Document to extract triples: <<docstart>> Steve Bright is the creator of Bananaman which was last broadcast on April 15th 1986 by the BBC <<docend>> 
 Extracted triples from the given document: <<triplestart>> 1. <subject> Steve_Bright | <relation> creator | <object
